In [1]:
#Importando a biblioteca
import pandas as pd
import tensorflow as tf

2026-02-12 16:04:12.145112: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-12 16:04:12.253171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-12 16:04:14.144334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV

In [3]:
#importando o arquivo de entradas
x = pd.read_csv('/home/Ront/Documentos/Card-14/Pratica/Seção 4/entradas_breast.csv') 
y = pd.read_csv('/home/Ront/Documentos/Card-14/Pratica/Seção 4/saidas_breast.csv') 

In [4]:
# Fiz com 30 pq minha maquina não é das melhores
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
    K.clear_session()
    
    modelo = Sequential([
        tf.keras.layers.InputLayer(shape=(30,)),
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    modelo.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
    return modelo

In [5]:
# Criar o wrapper do Keras para sklearn
modelo_keras = KerasClassifier(model=criar_rede)

In [6]:
# Definir os parâmetros para o GridSearch
parametros = {
    'batch_size': [5, 10],
    'epochs': [30],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'glorot_uniform'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16, 8]
}

In [7]:
# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=modelo_keras,
    param_grid=parametros,
    scoring='accuracy',
    cv=5,
    verbose=1
)

In [ ]:
# Executar o GridSearch
grid_search.fit(x, y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Epoch 1/30


2026-02-12 16:04:19.086639: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - binary_accuracy: 0.6088 - loss: 0.7803
Epoch 2/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6549 - loss: 0.5726
Epoch 3/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6396 - loss: 0.5414
Epoch 4/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7275 - loss: 0.4963
Epoch 5/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7121 - loss: 0.4742
Epoch 6/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8000 - loss: 0.4155
Epoch 7/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8044 - loss: 0.4341
Epoch 8/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8242 - loss: 0.4183
Epoch 9/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8352 - loss: 0.4247
Epoch 10/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8088 - loss: 0.3917
Epoch 11/30
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8703 - loss: 0.3219
Epoch 12

In [ ]:
# Exibir os melhores parâmetros encontrados
print(grid_search.best_params_)

In [ ]:
# Exibir a melhor acurácia
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

In [ ]:
#  Visualizar todos os resultados
resultados = pd.DataFrame(grid_search.cv_results_)
resultados[['params', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False)